In [11]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots, cm
import sklearn.model_selection as skm
from ISLP import load_data, confusion_table

In [12]:
from sklearn.svm import SVC
from ISLP.svm import plot as plot_svm
from sklearn.metrics import RocCurveDisplay

roc_curve = RocCurveDisplay.from_estimator

In [13]:
# Read in dataset
auto = pd.read_csv('Datasets/Auto.csv')
auto.info()
auto.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           392 non-null    float64
 1   cylinders     392 non-null    int64  
 2   displacement  392 non-null    float64
 3   horsepower    392 non-null    int64  
 4   weight        392 non-null    int64  
 5   acceleration  392 non-null    float64
 6   year          392 non-null    int64  
 7   origin        392 non-null    int64  
 8   name          392 non-null    object 
dtypes: float64(3), int64(5), object(1)
memory usage: 27.7+ KB


(392, 9)

In [14]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


#### Use the attached ‘auto.csv’ data file to build a support vector machines model to determine whether a given car gets high or low gas mileage.
1. Create a binary variable that takes on a value of 1 for cars with gas mileage above the median and 0 for cars with gas mileage below the median.

In [18]:
# Find the median mpg
median_mpg = auto['mpg'].median()

# Create new binary column using median mpg (22.75)
auto['high_mileage'] = (auto['mpg'] > median_mpg).astype(int)

# Verify new column
print(auto[['high_mileage', 'mpg']].head(10))
print(auto[['high_mileage', 'mpg']].tail(10))

   high_mileage   mpg
0             0  18.0
1             0  15.0
2             0  18.0
3             0  16.0
4             0  17.0
5             0  15.0
6             0  14.0
7             0  14.0
8             0  14.0
9             0  15.0
     high_mileage   mpg
382             1  26.0
383             0  22.0
384             1  32.0
385             1  36.0
386             1  27.0
387             1  27.0
388             1  44.0
389             1  32.0
390             1  28.0
391             1  31.0


2. Fit a support vector classifier to the data using a linear kernel. Use a seed of 123, if needed. Experiment with different values of C to predict whether a car gets high or low gas mileage.

3. Report the cross-validation errors associated with different values of this parameter. Discuss the empirical implications of those results.

4. Now repeat Q2, this time using SVMs with radial and polynomial basis kernels, with different values of gamma and degree and C. Discuss the empirical
implications of those results.

5. Generate relevant plots to support your findings in Q2 and Q4.